In [1]:
import os
import pandas as pd
from PIL import Image
import base64
import io


### Testing Tesseract

In [2]:
import pytesseract
from PIL import Image

# Set the path to the Tesseract executable (update this with your actual path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image_path):
    # Open the image using PIL (Pillow)
    image = Image.open(image_path)

    # Use Tesseract to extract text
    extracted_text = pytesseract.image_to_string(image)

    return extracted_text

if __name__ == "__main__":
    # Specify the path to your image file
    image_path = "C:/Users/Subhadeep/Downloads/uhc_images/Book-1.png"

    # Extract text from the image
    text = extract_text_from_image(image_path)

    # Print the extracted text
    print("Extracted Text:")
    print(text)


Extracted Text:
Group Name:

Aurora Products

Group Number (or TBD):

1264561

Request Date:

7214/2020

Effective Date of NSI

3/4/2020

Please Note: All items listed on this form are subject to review, Please highlight the NSB"

in yellow,

Benefits Currently in Place

Requested Benefits

Current Plan Current Plan 2: | Current Plan 3:
0100001 BUY UP Potoo02 core | 0100004 BASE
Base (Similar standard Plan) Tracking ID # (REQUIRED)
+ Reach out to your Underwriter
ia ta
# of Employees Enrolled: 39 2 25 28 26 18
Group State (NY, NJ, CT) cr cr ct cr cr cr
Market (Large or Small Group) Large Large Large Large Large Large
Product (e.g: HMO, POS, Classic, Access, Dect, OxUSA, Value Option
or EPO) HMO HMO HSA Wo HMO HSA
Access (Gated or Non-Gated) | Non-Gates Gated Non-Gated Non-Gated Gated Non-Gated
Network (Freedom, Liberty, or Choice Plus) Freedom Freedom Freedom Freedom Freedom Freedom
Current Carrier (Oxford or Competitor) Polaris Polaris Polaris Potaris Polaris Polaris
in Network (IN):


### Identifying item type in sheets

In [46]:
import os
import pathlib
import zipfile
from openpyxl import load_workbook
from PIL import Image
from openpyxl import load_workbook

def identify_sheet_content(sheet):
    has_images = any(sheet._images)
    has_linked_excel = any("excel" in rel.reltype for rel in sheet._rels)
    has_linked_pdf = any(cell.hyperlink.target.endswith('.pdf') for row in sheet.iter_rows() for cell in row if cell.hyperlink)

    has_text_and_numeric_data = any(cell.value for row in sheet.iter_rows() for cell in row if cell.data_type in ('n', 's'))

    if has_images:
        return 'image'
    elif has_linked_excel:
        return 'excel link'
    elif has_linked_pdf:
        return 'linked pdf'
    elif has_text_and_numeric_data:
        return 'tables'
    else:
        return 'Other'

def identify_excel_file(file_path):
    excel_file = load_workbook(file_path, data_only=True)
    sheet_types = {}

    for sheet_name in excel_file.sheetnames:
        sheet = excel_file[sheet_name]
        sheet_types[sheet_name] = identify_sheet_content(sheet)

    return sheet_types

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = 'C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx'
result = identify_excel_file(excel_file_path)

for sheet_name, content_type in result.items():
    print(f"Sheet '{sheet_name}': {content_type}")

Sheet 'group 1': image
Sheet 'group 2': tables
Sheet 'files': tables
Sheet 'Sheet4': tables
Sheet 'Sheet5': image
Sheet 'group 6': tables


In [55]:
from openpyxl.drawing.image import Image as xlImage

def identify_sheet_content(sheet):
    has_images = any(sheet._images)
    has_linked_pdf = any(cell.hyperlink.target.endswith('.pdf') for row in sheet.iter_rows() for cell in row if cell.hyperlink)
    has_text_and_numeric_data = any(cell.value for row in sheet.iter_rows() for cell in row if cell.data_type in ('n', 's'))

    # Check for linked Excel files in the form of objects
    has_linked_excel_objects = hasattr(sheet, '_drawing') and sheet._drawing is not None and any(obj.type == "Excel.Sheet" for obj in sheet._drawing.objects if isinstance(obj, xlImage))

    if has_images:
        return 'image'
    elif has_linked_pdf:
        return 'linked pdf'
    elif has_linked_excel_objects:
        return 'linked excel objects'
    elif has_text_and_numeric_data:
        return 'tables'
    else:
        return 'Other'


def identify_excel_file(file_path):
    excel_file = load_workbook(file_path, data_only=True)
    sheet_types = {}

    for sheet_name in excel_file.sheetnames:
        sheet = excel_file[sheet_name]
        sheet_types[sheet_name] = identify_sheet_content(sheet)

    return sheet_types

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = 'C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx'
result = identify_excel_file(excel_file_path)

for sheet_name, content_type in result.items():
    print(f"Sheet '{sheet_name}': {content_type}")



Sheet 'group 1': image
Sheet 'group 2': tables
Sheet 'files': Other
Sheet 'Sheet4': Other
Sheet 'Sheet5': image
Sheet 'group 6': tables


In [44]:
import openpyxl
import re

def check_external_links(workbook_path):
    """
    Check for external links in a workbook and return 'Excel Link' if found.

    Args:
        workbook_path (pathlib.Path or str): Excel file path.

    Returns:
        result (str): 'Excel Link' if external links are found, otherwise an empty string.
    """
    # Convert path to pathlib.Path if it's a string
    if isinstance(workbook_path, str):
        workbook_path = pathlib.Path(workbook_path)

    # Check if the file has the '.xlsx' extension
    if workbook_path.suffix != '.xlsx':
        raise ValueError('Path must be an xlsx file')

    # Open the workbook
    workbook = openpyxl.load_workbook(workbook_path, read_only=True)

    try:
        # Iterate through all sheets in the workbook
        for sheet_name in workbook.sheetnames:
            sheet = workbook[sheet_name]

            # Iterate through all cells in the sheet
            for row in sheet.iter_rows():
                for cell in row:
                    # Check if the cell formula contains an external link
                    formula = str(cell.formula)
                    if re.search(r'=\s*Excel\.[^\s]+\s*\|', formula):
                        return 'Excel Link'

    finally:
        # Close the workbook
        workbook.close()

    # Return an empty string if no external links are found
    return ''

# Example usage
excel_file_path = "C:/Users/Subhadeep/Downloads/testbook1.xlsx"
result = check_external_links(excel_file_path)
print(result)




### Extracting Images and storing them in a new directory

In [67]:
import os
import shutil
import pathlib
import zipfile

def extract_images_from_excel(path, output_folder_name='extracted_images'):
    """
    Extracts images from an Excel file and stores them in a single folder.

    Args:
        path (pathlib.Path or str): Excel file path.
        output_folder_name (str): Name of the folder to store the extracted images.
            Defaults to 'extracted_images'.

    Returns:
        new_paths (list[pathlib.Path]): List of paths to the extracted images.
    """
    # Convert path to pathlib.Path if it's a string
    if isinstance(path, str):
        path = pathlib.Path(path)

    # Check if the file has the '.xlsx' extension
    if path.suffix != '.xlsx':
        raise ValueError('Path must be an xlsx file')

    # Extract the filename (excluding the extension) using .stem
    name = path.stem

    # Create a new folder for the extracted images
    output_folder = path.parent / output_folder_name
    output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

    # Create a temporary directory for unzipping the Excel file
    temp_dir = path.parent / 'temp'
    temp_dir.mkdir(exist_ok=True)  # Create folder if it doesn't exist

    try:
        # Unzip the Excel file into the temporary directory
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Locate the 'media' directory within the unzipped content
        media_dir = temp_dir / 'xl' / 'media'

        image_index = 0  # Initialize an index for the images
        new_paths = []  # List to store the paths of the extracted images

        # Iterate through the files in the 'media' directory
        for root, dirs, files in os.walk(media_dir):
            for file in files:
                image_index += 1  # Increment the image index for each image found

                # Construct paths for the original image and the new destination
                image_path = pathlib.Path(root) / file
                new_path = output_folder / f'{name}-{str(image_index)}.png'

                # Copy the image to the output folder with a new name
                shutil.copy(image_path, new_path)

                # Store the new path in the list
                new_paths.append(new_path)

    finally:
        # Cleanup: Remove the temporary directory
        shutil.rmtree(temp_dir)

    # Return the list of paths to the extracted images
    return new_paths

In [68]:
path="C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx"
extract_images_from_excel(path, output_folder_name='mns_images')

[WindowsPath('C:/Users/Subhadeep/Downloads/mns_images/Sample MNS Bartrack Report-1.png'),
 WindowsPath('C:/Users/Subhadeep/Downloads/mns_images/Sample MNS Bartrack Report-2.png'),
 WindowsPath('C:/Users/Subhadeep/Downloads/mns_images/Sample MNS Bartrack Report-3.png'),
 WindowsPath('C:/Users/Subhadeep/Downloads/mns_images/Sample MNS Bartrack Report-4.png'),
 WindowsPath('C:/Users/Subhadeep/Downloads/mns_images/Sample MNS Bartrack Report-5.png')]

#### Extracting text data and storing them in a new excel sheet

In [73]:
import os
import shutil
import pathlib
import openpyxl

def extract_text_from_excel(input_path, output_folder_name='extracted_text'):
    """
    Extracts text data from each sheet of an Excel file and stores them in a new Excel file.

    Args:
        input_path (pathlib.Path or str): Excel file path.
        output_folder_name (str): Name of the folder to store the extracted text.
            Defaults to 'extracted_text'.

    Returns:
        output_path (pathlib.Path): Path to the new Excel file containing extracted text.
    """
    # Convert input_path to pathlib.Path if it's a string
    if isinstance(input_path, str):
        input_path = pathlib.Path(input_path)

    # Check if the file has the '.xlsx' extension
    if input_path.suffix != '.xlsx':
        raise ValueError('Path must be an xlsx file')

    # Create a new folder for the extracted text
    output_folder = input_path.parent / output_folder_name
    output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

    # Create a new Excel file for storing extracted text
    output_path = output_folder / f'{input_path.stem}_text.xlsx'
    wb_output = openpyxl.Workbook()

    try:
        # Open the original Excel file
        wb_input = openpyxl.load_workbook(input_path)

        # Iterate through each sheet
        for sheet_name in wb_input.sheetnames:
            ws_input = wb_input[sheet_name]

            # Create a new sheet in the output workbook
            ws_output = wb_output.create_sheet(title=sheet_name)

            # Iterate through each row and column in the sheet
            for row in ws_input.iter_rows(values_only=True):
                # Append the row data to the new sheet
                ws_output.append(row)

    finally:
        # Save the new Excel file
        wb_output.save(output_path)

    # Return the path to the new Excel file containing extracted text
    return output_path

def process_excel_file(input_path, output_folder_name='extracted_text'):
    """
    Process an Excel file by extracting text data from each sheet and storing them in a new Excel file.

    Args:
        input_path (pathlib.Path or str): Excel file path.
        output_folder_name (str): Name of the folder to store the extracted text.
            Defaults to 'extracted_text'.
    """
    # Extract text from the Excel file
    output_path = extract_text_from_excel(input_path, output_folder_name)

    # Print the path to the new Excel file
    print(f'Text data extracted and saved to: {output_path}')

# Example usage:
# Specify the input Excel file and the output folder name
excel_file_path = pathlib.Path('C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx')
output_folder_name = 'extracted_text'

# Process the Excel file
process_excel_file(excel_file_path, output_folder_name)


Text data extracted and saved to: C:\Users\Subhadeep\Downloads\extracted_text\Sample MNS Bartrack Report_text.xlsx


In [8]:
import openpyxl
import os

def extract_linked_files(input_excel_path, output_folder):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(input_excel_path, read_only=True, data_only=True)

    # Iterate through all sheets in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Check if the sheet has legacy drawings
        if sheet.legacy_drawing:
            # Iterate through legacy drawings (linked objects)
            for drawing in sheet.legacy_drawing:
                if drawing.prog_id == 'Excel.Sheet':
                    # Extract linked file path
                    linked_file_path = drawing.embedded
                    print(f"Linked file in sheet '{sheet_name}': {linked_file_path}")

                    # Extract the linked file content
                    linked_file_content = drawing.image.blob
                    output_file_path = os.path.join(output_folder, f"{sheet_name}_linked_file.xlsx")

                    # Save the linked file content to a new file
                    with open(output_file_path, 'wb') as output_file:
                        output_file.write(linked_file_content)

    # Close the workbook
    workbook.close()

# Example usage
input_excel_path = 'C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx'
output_folder = "C:/Users/Subhadeep/Downloads/testfolder"
extract_linked_files(input_excel_path, output_folder)





In [29]:
import os
import zipfile

def extract_embedded_files(file_path, save_path):
    """
    Extracts arbitrary embedded files from an Excel (xlsx) file and saves them.

    Parameters:
    ----------
    file_path : str,
        The path to the xlsx file.

    save_path : str,
        Directory path to save the extracted files.
    """
    with zipfile.ZipFile(file_path, 'r') as zip_file:
        # List all files in the archive
        all_files = zip_file.namelist()

        # Iterate through all files in the archive
        for file_info in all_files:
            # Extract the file content
            file_content = zip_file.read(file_info)

            # Save the extracted file to the specified directory
            extracted_file_path = os.path.join(save_path, os.path.basename(file_info))
            with open(extracted_file_path, 'wb') as f:
                f.write(file_content)

# Example usage:
xlsx_file_path = "C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx"
save_path = "C:/Users/Subhadeep/Downloads/extracted_excel_files/"
extract_embedded_files(xlsx_file_path, save_path)

### Extracting Embedded files

In [32]:
import os
import zipfile
import mimetypes
import io
import csv
from PIL import Image

def extract_embedded_files(file_path, save_path):
    """
    Extracts various types of embedded files from an Excel (xlsx) file and saves them.

    Parameters:
    ----------
    file_path : str,
        The path to the xlsx file.

    save_path : str,
        Directory path to save the extracted files.
    """
    # Validate file extension
    if not file_path.lower().endswith('.xlsx'):
        raise ValueError("Invalid file format. Please provide a valid Excel (xlsx) file.")

    # Create the directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)

    with zipfile.ZipFile(file_path, 'r') as zip_file:
        # List all files in the archive
        all_files = zip_file.namelist()

        # Iterate through all files in the archive
        for file_info in all_files:
            # Extract the file content
            file_content = zip_file.read(file_info)

            # Determine the type of file
            mime_type, _ = mimetypes.guess_type(file_info)

            # Save the extracted file based on its type
            extracted_file_path = os.path.join(save_path, os.path.basename(file_info))

            if mime_type and mime_type.startswith('image'):
                # Save images as image files
                with open(extracted_file_path, 'wb') as f:
                    f.write(file_content)

            elif mime_type and mime_type == 'text/csv':
                # Save text/tabular data as CSV files
                with open(extracted_file_path, 'wb') as f:
                    f.write(file_content)

            elif mime_type and mime_type.startswith('application/vnd.openxmlformats-officedocument.spreadsheetml'):
                # Save embedded XLSX files
                with open(extracted_file_path, 'wb') as f:
                    f.write(file_content)

            elif mime_type and mime_type.startswith('application') and 'Excel.Sheet' in mime_type:
                # Save other embedded Excel sheets
                with open(extracted_file_path, 'wb') as f:
                    f.write(file_content)

# Example usage:
xlsx_file_path = "C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx"
save_path = "C:/Users/Subhadeep/Downloads/extracted_embedded_files/"
extract_embedded_files(xlsx_file_path, save_path)



In [33]:
import os
import xml.etree.ElementTree as ET

def convert_files_to_txt(input_folder, output_folder):
    """
    Converts each file in the input folder to a plain text file and saves it in the output folder.

    Parameters:
    ----------
    input_folder : str,
        The path to the folder containing XML and RELS files.

    output_folder : str,
        The path to the folder where the converted text files will be saved.
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")

        try:
            with open(input_file_path, 'r', encoding='utf-8') as input_file:
                # Read the content of the input file
                file_content = input_file.read()

                # Optionally, you can parse XML content if needed
                # For example, if the file is in XML format
                # tree = ET.fromstring(file_content)
                # parsed_content = ET.tostring(tree, encoding='utf-8').decode('utf-8')

            # Write the content to the output text file
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                output_file.write(file_content)
        except Exception as e:
            print(f"Error processing file {input_file_path}: {e}")

# Example usage:
input_folder = "C:/Users/Subhadeep/Downloads/extracted_excel_files"
output_folder = "C:/Users/Subhadeep/Downloads/output folder"
convert_files_to_txt(input_folder, output_folder)

Error processing file C:/Users/Subhadeep/Downloads/extracted_excel_files\image1.jpeg: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Error processing file C:/Users/Subhadeep/Downloads/extracted_excel_files\image2.emf: 'utf-8' codec can't decode byte 0x80 in position 72: invalid start byte
Error processing file C:/Users/Subhadeep/Downloads/extracted_excel_files\image3.emf: 'utf-8' codec can't decode byte 0x80 in position 72: invalid start byte
Error processing file C:/Users/Subhadeep/Downloads/extracted_excel_files\image4.png: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte
Error processing file C:/Users/Subhadeep/Downloads/extracted_excel_files\printerSettings1.bin: 'utf-8' codec can't decode byte 0xdc in position 68: invalid continuation byte


In [38]:
import os
import re

import os
import re

def extract_text_from_folder(folder_path):
    """
    Extracts text content matching the pattern from text files within a folder.

    Parameters:
    ----------
    folder_path : str,
        The path to the folder containing text files.

    Returns:
    -------
    matching_paths : list,
        A list of concatenated file paths found within the text files.
    """
    matching_paths = []

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a text file
        if os.path.isfile(file_path) and filename.lower().endswith('.txt'):
            try:
                # Read the content of the text file
                with open(file_path, 'r', encoding='utf-8') as file:
                    file_content = file.read()

                # Use format() to build the string
                matches = re.findall(r'file:///C:\\Users\\([^"]+)', file_content)
                matching_paths.extend(["C:/Users/{}".format(match.replace('\\', '/')) for match in matches])
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    return matching_paths

# Example usage:
folder_path = "C:/Users/Subhadeep/Downloads/output folder"
matching_paths = extract_text_from_folder(folder_path)

if matching_paths:
    print("Matching paths:")
    for path in matching_paths:
        print(path)
else:
    print("No matching paths found in the folder.")

Matching paths:
C:/Users/Subhadeep/OneDrive/Documents/AMZN%20NOV%202020%20Puts%20(Interactive%20Data).csv
C:/Users/Subhadeep/OneDrive/Documents/AMZN%20NOV%202020%20Calls%20(Interactive%20Data).csv


### Identifying Item type

In [40]:
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as xlImage

def identify_sheet_content(sheet):
    has_images = any(sheet._images)
    has_linked_pdf = any(cell.hyperlink.target.endswith('.pdf') for row in sheet.iter_rows() for cell in row if cell.hyperlink)
    has_text_and_numeric_data = any(cell.value for row in sheet.iter_rows() for cell in row if cell.data_type in ('n', 's'))

    # Check for linked Excel files in the form of objects
    has_linked_excel_objects = hasattr(sheet, '_drawing') and sheet._drawing is not None and any(obj.type == "Excel.Sheet" for obj in sheet._drawing.objects if isinstance(obj, xlImage))

    if has_images and has_text_and_numeric_data:
        return 'text and image'
    elif has_images:
        return 'image'
    elif has_linked_pdf:
        return 'linked pdf'
    elif has_linked_excel_objects:
        return 'linked excel objects'
    elif has_text_and_numeric_data:
        return 'tables'
    else:
        return 'Other'

def identify_excel_file(file_path):
    excel_file = load_workbook(file_path, data_only=True)
    sheet_types = {}

    for sheet_name in excel_file.sheetnames:
        sheet = excel_file[sheet_name]
        sheet_types[sheet_name] = identify_sheet_content(sheet)

    return sheet_types

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = 'C:/Users/Subhadeep/Downloads/Sample MNS Bartrack Report.xlsx'
result = identify_excel_file(excel_file_path)

for sheet_name, content_type in result.items():
    print(f"Sheet '{sheet_name}': {content_type}")


Sheet 'group 1': image
Sheet 'group 2': tables
Sheet 'files': Other
Sheet 'Sheet4': Other
Sheet 'Sheet5': text and image
Sheet 'group 6': tables
